# Transfer Learning

# Imports

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import os
import sys


from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.utils import load_img, img_to_array

from sklearn.model_selection import cross_val_score, learning_curve, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss, accuracy_score, classification_report

from xgboost import XGBClassifier

from functools import partial
import tqdm.notebook as tq

import warnings
warnings.filterwarnings('ignore')

# Paths

In [2]:
VGG16_PATH = '/kaggle/input/vgg16/'
IMAGES_PATH = '/kaggle/input/dog-breed-identification/'
BATCH_SIZE = 100

# Helper functions

In [3]:
def path_given_id(id, test=False):
    """
    Returns the full path to the image given the id of the image.
    Parameters:
        - id: The id of the image.
        - test: If True returns the relative path from the test folder. Otherwise, returns the relative path to the image from the training folder.
    Returns:
        - The full relative path to the image with the give in id.
    """
    return IMAGES_PATH + ('train/' if not test else 'test/') + id + '.jpg'

def get_img_array(id, test=False):
    """
    Loads the image from the given id, convert the image to a numpy array and return the numpy array.
    Parameters:
        - id: The id of the image.
        - test: If True, loads the image from the test folder. If False,loads the image from the train folder.
    Returns:
        - The image with the give id as a numpy array.
    """
    img = load_img(path_given_id(id, test), target_size=(224, 224))
    return img_to_array(img)

# Read the labels provided in the CSV file

In [4]:
temp_labels = pd.read_csv(IMAGES_PATH + 'labels.csv')
labels = temp_labels.set_index('id')
labels = labels['breed']

y = pd.get_dummies(labels, sparse = True)
image_ids = y.index

# Create and instantiate the model
## Here we are downloading the pretrained weights from kaggle

In [5]:
# Here we are loading the no top version. This means that we are throwing away the fully connected layers
# and the softmax classifier at the end and we will build our own layers here.
vgg16_model = VGG16(weights=(VGG16_PATH + 'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                  include_top=False, pooling='avg')

# preprocess_input(np.expand_dims(get_image_array(id, test), axis=0)) will convert the image into 1,224,224,3 to give to predict.
def process_image(id, test=False):
    return vgg16_model.predict(preprocess_input(np.expand_dims(get_img_array(id, test), axis=0)))

In [6]:
X_intermediate = []

# The opening of the document and writing to a temporary file has no significance other than avoiding 
# cluttering up the notebook with single image outputs. It can be removed without any issues.
sys.stdout = open('output.txt', 'w')

for id in tq.tqdm(image_ids):
    X_intermediate.append(process_image(id)[0])

  0%|          | 0/10222 [00:00<?, ?it/s]

In [7]:
X= pd.DataFrame(X_intermediate, index=image_ids)
X.shape

(10222, 512)

In [8]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
id,,,,,,,,,,,,,,,,,,,,,
000bec180eb18c7604dcecc8fe0dba07,5.070187,3.962066,3.375381,0.000000,3.670384,1.779856,0.000000,3.211671,2.065429,1.705278,...,0.000000,0.661565,5.617185,0.398247,0.000000,0.000000,0.000000,3.253707,0.793703,2.560010
001513dfcb2ffafc82cccf4d8bbaba97,8.837939,0.000000,23.241722,1.163514,0.045581,0.235458,1.752838,4.239479,0.790429,0.109258,...,0.231802,0.101770,0.352381,14.032310,3.753132,0.101469,0.154369,9.209677,5.048140,0.000000
001cdf01b096e06d78e9e5112d419397,0.450912,1.027726,7.698472,0.252432,3.378808,0.332153,0.187855,0.106496,15.604666,3.175185,...,2.951074,1.198248,0.252030,1.587865,1.819199,1.838186,1.691979,0.471272,1.725393,0.209770
00214f311d5d2247d5dfe4fe24b2303d,1.059968,0.000000,13.055110,0.000000,4.934757,1.156982,4.262399,3.815343,2.995876,0.013436,...,0.438529,0.000000,0.072758,2.224409,4.519608,0.761705,0.000000,0.000000,0.551235,0.312393
0021f9ceb3235effd7fcde7f7538ed62,0.501041,0.000000,5.049969,0.000000,4.451237,3.013023,0.433602,0.000000,0.884072,2.204053,...,3.103924,0.039919,0.287888,0.565757,1.917303,0.166229,0.135676,0.606473,4.193654,0.988650


# Building our own layers at the end

In [9]:
final_model = Sequential([
    Dense(1024, input_shape=(512,)),
    Activation('relu'),
    Dense(256, input_shape=(512,)),
    Activation('relu'),
    Dense(120),
    Activation('softmax'),
])

final_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Split the data into training and testing sets

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=69)

In [11]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,502,503,504,505,506,507,508,509,510,511
id,,,,,,,,,,,,,,,,,,,,,
556daede719158bd83c4b775844d9bcb,1.419100,0.000000,5.335067,1.307954,5.726939,8.972469,0.000000,0.283493,1.520824,1.984347,...,0.026766,3.195892,1.993027,2.479052,0.260208,0.398424,3.709291,0.234797,0.844397,1.038007
19a845cb10c06885ee147501491a65c9,0.691485,0.000000,18.371063,0.000000,33.490658,1.263164,0.000000,1.107117,22.671600,0.251017,...,1.389739,0.440681,0.736947,2.178701,5.106574,0.045755,0.794699,0.259826,1.885160,3.629787
181e91cb6caf6739478d06231faa053d,3.465265,17.515654,9.139524,7.594967,12.042945,2.522082,0.000000,0.414698,9.338680,0.055067,...,2.014037,1.377743,0.480559,17.901056,1.967806,5.097197,1.452625,14.936264,1.312257,28.365129
79a7fb5f7c40be0c75331a0a8558191d,0.000000,4.020119,8.490611,0.008008,25.405817,5.769643,0.246865,0.036183,20.416828,0.000000,...,0.478695,1.666424,0.735093,10.826945,0.000000,0.000000,0.093343,0.033495,0.059183,6.274520
864dd68d1079dd10a784dbd2ff2c3995,0.000000,2.306849,3.302875,1.441267,9.369179,2.931309,2.411290,1.067827,2.543838,0.870693,...,1.054701,1.667845,0.024694,3.590480,0.190636,0.029709,6.296655,3.398437,4.533296,9.534866


In [12]:
y_train.head()

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,...,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
id,,,,,,,,,,,,,,,,,,,,,
556daede719158bd83c4b775844d9bcb,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
19a845cb10c06885ee147501491a65c9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
181e91cb6caf6739478d06231faa053d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
79a7fb5f7c40be0c75331a0a8558191d,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
864dd68d1079dd10a784dbd2ff2c3995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Fit the model

In [13]:
final_model.fit(X_train, np.asarray(y_train), epochs=150, batch_size=BATCH_SIZE, verbose=False)

# Make predictions

In [14]:
y_pred = pd.DataFrame(final_model.predict(X_test))

y_pred = y_pred.idxmax(axis=1)
y_pred

0        66
1       100
2        27
3        99
4        66
       ... 
2040     69
2041    117
2042     56
2043     35
2044     66
Length: 2045, dtype: int64

In [15]:
preds = []
for i in range(len(y_pred)):
    preds.append(y.columns[y_pred[i]])    
    
preds[:10]

['kuvasz',
 'shih-tzu',
 'cardigan',
 'shetland_sheepdog',
 'kuvasz',
 'basenji',
 'maltese_dog',
 'dandie_dinmont',
 'boxer',
 'keeshond']

In [16]:
testing_labels = list(y_test.idxmax(axis=1))
testing_labels[:10]

['english_setter',
 'shih-tzu',
 'cardigan',
 'shetland_sheepdog',
 'kuvasz',
 'basenji',
 'lhasa',
 'dandie_dinmont',
 'american_staffordshire_terrier',
 'norwegian_elkhound']

# Calculating the accuracy

In [17]:
correct_predictions_count = 0
for i in range(len(testing_labels)):
    if testing_labels[i] == preds[i]:
        correct_predictions_count += 1
        
f"Accuracy of the model using the above data is {correct_predictions_count / len(testing_labels) * 100 :.2f}%"

'Accuracy of the model using the above data is 64.21%'